In [1]:
import pandas as pd
import seaborn as sn
from sklearn.preprocessing import LabelEncoder
import spacy
import re
import numpy as np
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras.layers import Embedding,Dense,LSTM,Dropout
import keras_tuner as kt
from tensorflow.keras.regularizers import l1_l2
from sklearn.preprocessing import MinMaxScaler 
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [2]:
a=pd.read_excel("/kaggle/input/sa-xlsx/sa.xlsx")
a.head()

,Year,Month,Day,Time of Tweet,text,sentiment,Platform
0,2018,8,18,noon,"I feel sorry, I miss you here in the sea beach",positive,FaceBook
1,2022,6,8,morning,We attend in the class just for listening teac...,negative,FaceBook
2,2017,12,28,morning,2 am feedings for the baby are fun when he is ...,positive,FaceBook
3,2018,5,28,morning,Today first time I arrive in the boat. Its ama...,positive,FaceBook
4,2016,5,15,morning,My Sharpie is running DANGERously low on ink,negative,FaceBook


In [3]:
a.shape

(499, 7)

In [4]:
a.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 499 entries, 0 to 498
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Year           499 non-null    int64 
 1   Month          499 non-null    int64 
 2   Day            499 non-null    int64 
 3   Time of Tweet  499 non-null    object
 4   text           499 non-null    object
 5   sentiment      499 non-null    object
 6   Platform       499 non-null    object
dtypes: int64(3), object(4)
memory usage: 27.4+ KB


In [5]:
a["Platform"].value_counts()

Platform
 Instagram     171
FaceBook       169
X              159
Name: count, dtype: int64

In [6]:
a["sentiment"].value_counts()

sentiment
neutral     199
positive    166
negative    134
Name: count, dtype: int64

In [7]:
a["text"][25]

'We attend in the class just for listening teachers reading on slide. Just Nonsence  '

In [8]:
a["text"][100]

'Watching Ellen Love her!! then doing the dishes  and tanning ...it`s gorgeous out!'

In [9]:
a["text"][300]

" the 'no pants' idea could be the new attempt world-wide to attract business back to the airlines."

In [10]:
a.drop(["Year","Month","Day","Time of Tweet","Platform"],axis=1,inplace=True)
a.head()

,text,sentiment
0,"I feel sorry, I miss you here in the sea beach",positive
1,We attend in the class just for listening teac...,negative
2,2 am feedings for the baby are fun when he is ...,positive
3,Today first time I arrive in the boat. Its ama...,positive
4,My Sharpie is running DANGERously low on ink,negative


In [11]:
le=LabelEncoder()
a["sentiment"]=le.fit_transform(a["sentiment"])
a.head()

,text,sentiment
0,"I feel sorry, I miss you here in the sea beach",2
1,We attend in the class just for listening teac...,0
2,2 am feedings for the baby are fun when he is ...,2
3,Today first time I arrive in the boat. Its ama...,2
4,My Sharpie is running DANGERously low on ink,0


In [12]:
nlp=spacy.load("en_core_web_sm")
def text_pp(t):
  t=t.lower()
  t=t.replace("`","").strip()
  t=t.replace("''","").strip()
  t=re.sub(r'\s+', ' ', t).strip()
  t=nlp(t)
  p=[w.lemma_ for w in t if not w.is_stop and not w.is_punct]
  return ' '.join(p)
a["finaltext"]=a["text"].apply(text_pp)
a.head()

,text,sentiment,finaltext
0,"I feel sorry, I miss you here in the sea beach",2,feel sorry miss sea beach
1,We attend in the class just for listening teac...,0,attend class listen teacher read slide nonsence
2,2 am feedings for the baby are fun when he is ...,2,2 feeding baby fun smile coo
3,Today first time I arrive in the boat. Its ama...,2,today time arrive boat amazing journey
4,My Sharpie is running DANGERously low on ink,0,sharpie run dangerously low ink


In [13]:
a.drop(["text"],axis=1,inplace=True)
a.head()

,sentiment,finaltext
0,2,feel sorry miss sea beach
1,0,attend class listen teacher read slide nonsence
2,2,2 feeding baby fun smile coo
3,2,today time arrive boat amazing journey
4,0,sharpie run dangerously low ink


In [14]:
a["finaltext"][25]

'attend class listen teacher read slide nonsence'

In [15]:
a["finaltext"][100]

'watch ellen love dish tanning gorgeous'

In [16]:
a["finaltext"][300]

'pant idea new attempt world wide attract business airline'

In [17]:
tok=Tokenizer(num_words=1000, oov_token="<OOV>")
tok.fit_on_texts(a["finaltext"])
seq=tok.texts_to_sequences(a["finaltext"])
pad=pad_sequences(seq,maxlen=18,padding="post")
print(pad[0])

[ 16  28  14 173 174   0   0   0   0   0   0   0   0   0   0   0   0   0]


In [18]:
x_train,x_test,y_train,y_test=train_test_split(pad,a["sentiment"],test_size=0.1,random_state=42)
x_train.shape

(449, 18)

In [19]:
model=Sequential([Embedding(input_dim=1170,output_dim=80,input_length=10),
                  LSTM(64,kernel_regularizer=l1_l2(l1=1e-4,l2=1e-4)),
                  Dropout(0.4),
                  Dense(3,activation="softmax")])
model.compile(optimizer=Adam(0.001),loss="sparse_categorical_crossentropy",metrics=["accuracy"])
model.fit(x_train,y_train,epochs=15)

Epoch 1/15


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


15/15 ━━━━━━━━━━━━━━━━━━━━ 3s 11ms/step - accuracy: 0.4057 - loss: 1.2333
Epoch 2/15
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.4362 - loss: 1.1899
Epoch 3/15
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.4070 - loss: 1.1808
Epoch 4/15
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.3954 - loss: 1.1526
Epoch 5/15
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.4436 - loss: 1.0762
Epoch 6/15
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.6558 - loss: 0.8280
Epoch 7/15
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.7044 - loss: 0.6977
Epoch 8/15
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.8540 - loss: 0.4763
Epoch 9/15
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.8853 - loss: 0.3502
Epoch 10/15
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.9769 - loss: 0.1504
Epoch 11/15
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.9618 - loss: 0.2079
Epoch 12/15
15/15 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.9500 - l

In [20]:
ls,acc=model.evaluate(x_test,y_test)

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.6854 - loss: 0.8800  
